取得套件

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

拿取train 資料

In [32]:
normalized_train = pd.read_csv('../result/all_beta_normalized_train_035.csv')

normalized_test = pd.read_csv('../result/all_beta_normalized_test_035.csv')

normalized_train_c = normalized_train[normalized_train['Unnamed: 0'].isin(normalized_test['Unnamed: 0'])]

X_train = normalized_train_c.iloc[:, 1::2]

將資料作為每個人的特徵

In [33]:
x = []

for i in range(445):
    X = X_train.iloc[:, i]

    X = X.values.flatten().tolist()

    x.append(X)

print(x[1])

y = [ (0 if i < 47 else 1)  for i in range(445)]

x = np.array(x)
y = np.array(y)


print(type(x),'x')
print(type(y),'y')


[0.07964195547395, 0.0306162399185655, 0.261522527187986, 0.0317381601785272, 0.0661343516750564, 0.034124742346744, 0.0612473721093203, 0.0405033425088478, 0.0421166306695464, 0.0441721288116272, 0.0588695788320025, 0.100177409816677, 0.294601927214894, 0.0876644736842105, 0.0261368291683736, 0.0257815419105742, 0.101882377964479, 0.0357238069848041, 0.117163321248563, 0.150895679662803, 0.0335174252523329, 0.0537719125467796, 0.0165824835604689, 0.859179019384265, 0.0384339448601004, 0.111423626135872, 0.0458655804480652, 0.0535502457210642, 0.0518960766303635, 0.0425753840318604, 0.048770424162403, 0.0380732015145141, 0.0406645177287379, 0.18425676467143, 0.0516438973127158, 0.0170070862859525, 0.0338581183611533, 0.0476667780376434, 0.0346178372961647, 0.0306251828733054, 0.0335010891609705, 0.0288745488351745, 0.0267751073745774, 0.0240879120879121, 0.0305685358255452, 0.0302554352318757, 0.12283056860677, 0.0457521374104598, 0.0245553145336226, 0.069061067888735, 0.09800873049094

開始預測

In [34]:

# 建立 Random Forest Classifier 模型
randomForestModel = RandomForestClassifier(n_estimators=100, criterion = 'gini')
# 使用訓練資料訓練模型

X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.2,random_state=0)

randomForestModel.fit(X_train, y_train)
# 使用訓練資料預測分類
predicted = randomForestModel.predict(X_train)
predicted_t = randomForestModel.predict(X_test)

print('訓練集: ',randomForestModel.score(X_train,y_train))
print('測試集: ',randomForestModel.score(X_test,y_test))

# 預測成功的比例


訓練集:  1.0
測試集:  0.9775280898876404


將特徵重要程度輸出並匯出

In [35]:

print('特徵重要程度: ',randomForestModel.feature_importances_)

df = pd.DataFrame({
    'Probe_ID': normalized_train['Unnamed: 0'],
    'import_level_in_forest': randomForestModel.feature_importances_
})

# print(df)

# df.to_csv("../result/imp_level.csv",index=False)

特徵重要程度:  [0.00000000e+00 9.85877584e-03 0.00000000e+00 0.00000000e+00
 3.45765346e-04 0.00000000e+00 2.09380040e-03 1.12693623e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 6.77503130e-03
 0.00000000e+00 2.82679901e-04 1.73679375e-04 0.00000000e+00
 8.02336810e-03 9.29714486e-03 0.00000000e+00 4.19910356e-04
 0.00000000e+00 1.19614037e-03 0.00000000e+00 9.08839607e-04
 3.05930629e-04 0.00000000e+00 6.62333271e-03 2.77671369e-04
 1.28216065e-03 0.00000000e+00 2.10859538e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.57799445e-03 1.30518379e-03
 1.20237264e-03 7.78953181e-03 0.00000000e+00 7.35062589e-04
 3.15319804e-04 0.00000000e+00 0.00000000e+00 4.60874568e-04
 0.00000000e+00 6.95614416e-04 1.52098104e-03 1.63003663e-03
 0.00000000e+00 2.43594382e-03 0.00000000e+00 3.27035307e-04
 0.00000000e+00 5.95549502e-02 0.00000000e+00 1.18553730e-03
 0.00000000e+00 1.37187447e-05 9.83457668e-03 0.00000000e+00
 0.00000000e+00